# Bias-correction for weather forecasts

We will use IBM PAIRS to study and correct biases in the GFS forecast. In detail, we will compare the temperature forecast made by NOAA's GFS with the ERA5 reanalysis. Aggregating 5 years worth of data will give evidence of a systematic yet location-dependent bias in the GFS data. We will then use PAIRS to remove these biases from the GFS forecast, leading to an MAE reduction of roughly 50%.

For details regarding both data layers, consult the PAIRS data explorer at https://ibmpairs.mybluemix.net/data-explorer. The layer IDs are
- 49423 (ERA5)
- 50195 (GFS)

In [ ]:
from datetime import datetime, timedelta
import os, numpy as np, pandas as pd
from matplotlib import pyplot as plt
from ibmpairs import paw

To start, we set some global variables. I.e. the name of your user account et cetera. Note that this assumes the existence of a file `ibmpairspass.txt` in your home directory containing access credentials for IBM PAIRS.

In [ ]:
PAIRS_USER              = '<username>'
PAIRS_SERVER            = 'https://pairs.res.ibm.com'
BASE_URI                = '/'
PAIRS_PASSWORD          = paw.get_pairs_api_password(PAIRS_SERVER, PAIRS_USER, passFile=os.path.expanduser('~/ibmpairspass.txt'))
PAIRS_CREDENTIALS       = (PAIRS_USER, PAIRS_PASSWORD)

The following helps when converting `datetime` objects to strings in ISO 8601-compliant format.

In [ ]:
iso8601 = '%Y-%m-%dT%H:%M:%SZ'

## Step 1: Temporal joins between the GFS and ERA5 data

Since the GFS data is a forecast, it does not only depend on a single timestamp. Indeed, a forecast is characterized both by its *valid time* and *issue time*. I.e. the time the forecast is for and the time it was issued. The difference between these is known as the *lead time* or *horizon*. I.e. `horizon = valid time - issue time`. Most forecasts in PAIRS are stored by issue time, simplifying the comparison with observations. The horizon is then an additional dimension, usually in hours. (For details consult the data explorer.)

To keep things simple, we will only consider the GFS data at horizon 6 h. I.e. predictions which were made 6 hours into the future. (Since the GFS predicts up to 15 days ahead, the maximum horizon is 360 hours.) The forecast is issued daily, which means that as long as we keep the horizon fixed at 6 we have one value every 24 hours. (To obtain data at higher frequencies, we could query additional horizons.)

The ERA5 data on the other hand is hourly. To join these two, we have to ensure that we only request timestamps for which both are defined. (The situation would be a bit different for a parameter which is accumulcated over a certain amount of time such as precipitation. Since temperature is generally considered an instantaneous quantity, we can simply join identical timestamps. Details on whether a parameter is instantaneous or defined over a certain measurement interval can be found in the data explorer.

Thus, we make a point query to both data layers to find all timestamps during the years 2014 to 2018.

In [ ]:
pointQueryJson = {
    'layers' : [
        {
            'type' : 'raster', 'id' : '50195', 'dimensions' : [{'name' : 'horizon', 'value' : '6'}]
        },
        {
            'type' : 'raster', 'id' : '49423'
        }
    ],
    'spatial' : {'type' : 'point', 'coordinates' : ['40', '-100']},
    'temporal' : {'intervals' : [{
        'start' : (datetime(2014, 1, 1) - timedelta(seconds = 1)).strftime(iso8601),
        'end' : datetime(2019, 1, 1).strftime(iso8601)
    }]}
}

In [ ]:
pointQuery = paw.PAIRSQuery(pointQueryJson, PAIRS_SERVER, PAIRS_CREDENTIALS)
pointQuery.submit()
pointQuery.vdf['value'] = pd.to_numeric(pointQuery.vdf['value'])

Some simple *pandas* transformations allow us to identify timestamps for which both layers are defined.

**Note** we can use this simple point query since both the forecast and reanalysis data is defined for all locations in the world at the same time. I.e. if we find data at a single time and point, we know that there will be data everywhere else in the world at that timestamp. This would not be the case were we to compare satellite data from e.g. Sentinel or Landsat satellites.

In [ ]:
pointQuery.vdf.pivot_table(index = 'timestamp', columns = 'layerId', values = 'value').dropna().head()

In [ ]:
completeDates = pointQuery.vdf.pivot_table(index = 'timestamp', columns = 'layerId', values = 'value').dropna().index.to_series(keep_tz = True)

## Step 2a: Identifying the overall bias

To proceed, we use the timestamps stored in `completeDates` to compare the GFS and ERA5 data at those timestamps. Since taking the mean and subtraction are interchangable operations, we take the mean for each dataset before calculating the bias. (Simply taking the average of all values in 2014-2018 would have led to wildly inconsistent results. At fixed horizon, the GFS layer only contains values for 0:00 UTC while the ERA5 one contains hourly data. Thus we would have compared aggregates over 0:00 UTC with those over all hours of the day.)

In [ ]:
biasQueryJson = {
    'layers' : [
        {
            'alias' : 'gfs',
            'type' : 'raster', 'id' : '50195', 'dimensions' : [{'name' : 'horizon', 'value' : '6'}],
            'aggregation' : 'Mean',
            'temporal' : {'intervals' : [{'snapshot' : ts.strftime(iso8601)} for _, ts in completeDates.iteritems()]},
            'output' : False
        },
        {
            'alias' : 'era5',
            'type' : 'raster', 'id' : '49423', 'aggregation' : 'Mean',
            'temporal' : {'intervals' : [{'snapshot' : ts.strftime(iso8601)} for _, ts in completeDates.iteritems()]},
            'output' : False
        },
        {
            'alias' : 'bias',
            'expression' : '$gfs - $era5'
        }
    ],
    'spatial' : {'type' : 'square', 'coordinates' : ['-90', '-170', '90', '170']},
    'temporal' : {'intervals' : [{
        'start' : (datetime(2015, 1, 1) - timedelta(seconds = 1)).strftime(iso8601),
        'end' : datetime(2016, 1, 1).strftime(iso8601)
    }]}
}

In [ ]:
biasQuery = paw.PAIRSQuery(biasQueryJson, PAIRS_SERVER, PAIRS_CREDENTIALS)
biasQuery.submit()

Note: The queries in this notebook are somewhat substantial and should take 5-10 minutes to complete. Calling `.poll_till_finished()` will thus block the notebook for an extended time. It's worth while to check the status of the query by calling `.poll()` and checking the `.queryStatus` object:

In [ ]:
biasQuery.poll()
biasQuery.queryStatus.json()['status']

In [ ]:
biasQuery.poll_till_finished()
biasQuery.download()
biasQuery.create_layers()

At first sight, we find fairly clear evidence of systematic bias. Temperatures in North- and South-America are generally predicted as too low while those in North and South-Africa appear to be too high.

In [ ]:
plt.figure(figsize = (20, 8))
plt.imshow(
    biasQuery.data['Expression-bias[bias]-Exp'],
    vmin = -5, vmax = 5, cmap = 'seismic',
    extent = [biasQuery.metadata['Expression-bias[bias]-Exp']['details']['boundingBox'][l] for l in ['minLongitude', 'maxLongitude', 'minLatitude', 'maxLatitude']]
)
plt.colorbar(label = 'Error [K]')
plt.title('Mean bias in GFS data 2014-2018')
plt.savefig('MeanBiasInGFS2014-2018.png', dpi = 60, bbox_inches = 'tight')
plt.show()

Having said that, it is instructive to look at the overall error distribution. Accumulating spatially leads to what is an essentially neglegible bias.

In [ ]:
pd.Series(biasQuery.data['Expression-bias[bias]-Exp'].reshape(-1)).dropna().hist(bins = 25, log = True)
plt.show()

In [ ]:
pd.Series(biasQuery.data['Expression-bias[bias]-Exp'].reshape(-1)).dropna().describe()

Finally, we note an mean absolute error (MAE) of about 0.86 degrees Kelvin.

In [ ]:
pd.Series(biasQuery.data['Expression-bias[bias]-Exp'].reshape(-1)).dropna().abs().mean()

## Step 2b: Temporal dependence of the biases

One might ask whether the bias we identified in the previous query is a systematic bias or an artifact of the aggregation. So let us take a further look. The following query calculates the bias for each year in the 5-year period independently.

In [ ]:
annualLayers = [
    [
    {
        'alias' : 'gfs_{}'.format(year),
        'type' : 'raster', 'id' : '50195', 'dimensions' : [{'name' : 'horizon', 'value' : '6'}],
        'aggregation' : 'Mean',
        'temporal' : {'intervals' : [{'snapshot' : ts.strftime(iso8601)} for _, ts in completeDates.iteritems() if ts.year == year]},
        'output' : False
    },
    {
        'alias' : 'era5_{}'.format(year),
        'type' : 'raster', 'id' : '49423', 'aggregation' : 'Mean',
        'temporal' : {'intervals' : [{'snapshot' : ts.strftime(iso8601)} for _, ts in completeDates.iteritems() if ts.year == year]},
        'output' : False
    },
    {
        'alias' : 'bias_{}'.format(year),
        'expression' : '$gfs_{year} - $era5_{year}'.format(year = year),
    }
    ] for year in range(2014, 2019)
]
annualLayers = [ll for l in annualLayers for ll in l]

annualBiasQueryJson = {
    'layers' : annualLayers,
    'spatial' : {'type' : 'square', 'coordinates' : ['-90', '-170', '90', '170']},
    'temporal' : {'intervals' : [{
        'start' : (datetime(2015, 1, 1) - timedelta(seconds = 1)).strftime(iso8601),
        'end' : datetime(2016, 1, 1).strftime(iso8601)
    }]}
}

In [ ]:
annualBiasQuery = paw.PAIRSQuery(annualBiasQueryJson, PAIRS_SERVER, PAIRS_CREDENTIALS)
annualBiasQuery.submit()

In [ ]:
annualBiasQuery.poll()
annualBiasQuery.queryStatus.json()['status']

In [ ]:
annualBiasQuery.poll_till_finished()
annualBiasQuery.download()
annualBiasQuery.create_layers()

To keep the plot simple we only show data for 2014 - 2017. The situation is essentially the same for 2018.

In [ ]:
extent = extent = [annualBiasQuery.metadata['Expression-bias_2014[bias_2014]-Exp']['details']['boundingBox'][l] for l in ['minLongitude', 'maxLongitude', 'minLatitude', 'maxLatitude']]
fig, ax = plt.subplots(2, 2, figsize = (30, 16), sharex = True, sharey = True)
ax[0][0].imshow(annualBiasQuery.data['Expression-bias_2014[bias_2014]-Exp'], vmin = -5, vmax = 5, cmap = 'seismic', extent = extent)
ax[0][1].imshow(annualBiasQuery.data['Expression-bias_2015[bias_2015]-Exp'], vmin = -5, vmax = 5, cmap = 'seismic', extent = extent)
ax[1][0].imshow(annualBiasQuery.data['Expression-bias_2016[bias_2016]-Exp'], vmin = -5, vmax = 5, cmap = 'seismic', extent = extent)
ax[1][1].imshow(annualBiasQuery.data['Expression-bias_2017[bias_2017]-Exp'], vmin = -5, vmax = 5, cmap = 'seismic', extent = extent)
ax[0][0].set_title('2014')
ax[0][1].set_title('2015')
ax[1][0].set_title('2016')
ax[1][1].set_title('2017')
plt.tight_layout()
plt.savefig('TemporalDependenceOfBias.png', dpi = 60, bbox_inches = 'tight')
plt.show()

As we can see, the biases are fairly consistent with time. We see differences in the details between the above aggregation periods, but overall structures are the same. I.e. too low predictions in the Americas, Europe and at the North Pole, too high predictions in North and South Africa, Northern and Central Asia as well as Antarctica.

## Step 3: A bias corrected forecast

Having confirmed the existence of a bias, we can now issue a bias-corrected forecast for 2019. I.e. we use the bias measured during the previous 5-year period to shift the predictions for 2019. To start, we again make a point query to facilitat the temporal join.

In [ ]:
pointQuery2019Json = {
    'layers' : [
        {
            'type' : 'raster', 'id' : '50195', 'dimensions' : [{'name' : 'horizon', 'value' : '6'}]
        },
        {
            'type' : 'raster', 'id' : '49423'
        }
    ],
    'spatial' : {'type' : 'point', 'coordinates' : ['40', '-100']},
    'temporal' : {'intervals' : [{
        'start' : (datetime(2019, 1, 1) - timedelta(seconds = 1)).strftime(iso8601),
        'end' : datetime(2019, 7, 1).strftime(iso8601)
    }]}
}

In [ ]:
pointQuery2019 = paw.PAIRSQuery(pointQuery2019Json, PAIRS_SERVER, PAIRS_CREDENTIALS)
pointQuery2019.submit()
pointQuery2019.vdf['value'] = pd.to_numeric(pointQuery2019.vdf['value'])

In [ ]:
completeDates2019 = pointQuery2019.vdf.pivot_table(index = 'timestamp', columns = 'layerId', values = 'value').dropna().index.to_series(keep_tz = True)

The following query again calculates the forecast bias of the years 2014-2018 and applies it to forecasts for the first half of 2019. Subsequently we calculate the bias of both the raw and bias-corrected 2019 forecasts.

In [ ]:
biasCorrectedQueryJson = {
    'layers' : [
        {
            'alias' : 'historic_gfs',
            'type' : 'raster', 'id' : '50195', 'dimensions' : [{'name' : 'horizon', 'value' : '6'}],
            'aggregation' : 'Mean',
            'temporal' : {'intervals' : [{'snapshot' : ts.strftime(iso8601)} for _, ts in completeDates.iteritems()]},
            'output' : False
        },
        {
            'alias' : 'historic_era5',
            'type' : 'raster', 'id' : '49423', 'aggregation' : 'Mean',
            'temporal' : {'intervals' : [{'snapshot' : ts.strftime(iso8601)} for _, ts in completeDates.iteritems()]},
            'output' : False
        },
        {
            'alias' : 'gfs',
            'type' : 'raster', 'id' : '50195', 'dimensions' : [{'name' : 'horizon', 'value' : '6'}],
            'aggregation' : 'Mean',
            'temporal' : {'intervals' : [{'snapshot' : ts.strftime(iso8601)} for _, ts in completeDates2019.iteritems()]},
            'output' : False
        },
        {
            'alias' : 'era5',
            'type' : 'raster', 'id' : '49423', 'aggregation' : 'Mean',
            'temporal' : {'intervals' : [{'snapshot' : ts.strftime(iso8601)} for _, ts in completeDates2019.iteritems()]},
            'output' : False
        },
        {
            'alias' : 'bias',
            'expression' : '$gfs - $era5',
            'output' : True
        },
        {
            'alias' : 'bias_corrected_forecast',
            'expression' : '$gfs - $historic_gfs + $historic_era5',
            'output' : True
        },
        {
            'alias' : 'bias_of_bias_corrected_forecast',
            'expression' : '$gfs - $historic_gfs + $historic_era5 - $era5',
            'output' : True
        }
    ],
    'spatial' : {'type' : 'square', 'coordinates' : ['-90', '-170', '90', '170']},
    'temporal' : {'intervals' : [{
        'start' : (datetime(2015, 1, 1) - timedelta(seconds = 1)).strftime(iso8601),
        'end' : datetime(2016, 1, 1).strftime(iso8601)
    }]}
}

In [ ]:
biasCorrectedQuery = paw.PAIRSQuery(biasCorrectedQueryJson, PAIRS_SERVER, PAIRS_CREDENTIALS)
biasCorrectedQuery.submit()

In [ ]:
biasCorrectedQuery.poll_till_finished()
biasCorrectedQuery.download()
biasCorrectedQuery.create_layers()

## Step 4: Results

We can finally analyze the impact of the bias correction. To do so we both plot the spatial distribution of biases but also calculate a number of global metrics. To start, let's take a look at the spatial distribution.

In [ ]:
extent = extent = [biasCorrectedQuery.metadata['Expression-bias[bias]-Exp']['details']['boundingBox'][l] for l in ['minLongitude', 'maxLongitude', 'minLatitude', 'maxLatitude']]
fig, ax = plt.subplots(2, 1, figsize = (30, 16), sharex = True, sharey = True)
ax[0].imshow(
    biasCorrectedQuery.data['Expression-bias[bias]-Exp'], vmin = -5, vmax = 5, cmap = 'seismic', extent = extent
)
ax[1].imshow(
    biasCorrectedQuery.data['Expression-bias_of_bias_corrected_forecast[bias_of_bias_corrected_forecast]-Exp'], vmin = -5, vmax = 5, cmap = 'seismic', extent = extent
)
ax[0].set_title('Bias of raw forecast')
ax[1].set_title('Bias of bias-corrected forecast')
plt.tight_layout()
plt.savefig('PerformanceOfBiasCorrectedForecast.png', dpi = 60, bbox_inches = 'tight')
plt.show()

Since the two plots above are on the same color scale (see the use of `vmin` and `vmax`), we can make direct comparisons from the colors. Clearly, the bias corrected forecast shows an overall improvement. Note that the underprediction in the Americas, the Indian subcontinent and Europe has turned into an overprediction. The situation is somewhat more complicated in the rest of the world though.

To complement this impression we calculate some general metrics. Note that the mean absolute error (MAE) has improved quite significantly.

In [ ]:
pd.DataFrame({
    'Raw forecast' : pd.Series(biasCorrectedQuery.data['Expression-bias[bias]-Exp'].reshape(-1)).dropna().describe().append(pd.Series({'MAE' : pd.Series(biasCorrectedQuery.data['Expression-bias[bias]-Exp'].reshape(-1)).dropna().abs().mean()})),
    'Bias-corrected forecast' : pd.Series(biasCorrectedQuery.data['Expression-bias_of_bias_corrected_forecast[bias_of_bias_corrected_forecast]-Exp'].reshape(-1)).dropna().describe().append(pd.Series({'MAE' : pd.Series(biasCorrectedQuery.data['Expression-bias_of_bias_corrected_forecast[bias_of_bias_corrected_forecast]-Exp'].reshape(-1)).dropna().abs().mean()}))
}).round(2)